In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Cargamos las tablas tanto de train como de test

In [124]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [125]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
train_data['Age'].dtype

In [ ]:
women = train_data.loc[train_data.Embarked == 'S']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [ ]:
men = train_data.loc[train_data.Embarked == 'C']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

Creamos una nueva columna "Relatives" formado por la unión de las columnas "SibSp" y "Parch"

In [126]:
train_data["Relatives"] = train_data["SibSp"]+train_data["Parch"]
train_data.head()

In [127]:
test_data["Relatives"] = test_data["SibSp"] + test_data["Parch"]
test_data.head()

Calculamos la media de edades para después sustituir los NaN por la media

In [128]:
colunmAgeTrain = train_data["Age"]
mediaAgeTrain = np.mean(colunmAgeTrain)
print(mediaAgeTrain)

In [129]:
colunmAgeTest = test_data["Age"]
mediaAgeTest = np.mean(colunmAgeTest)
print(mediaAgeTest)

In [130]:
train_data['Age'] = train_data['Age'].replace({np.nan:mediaAgeTrain})
train_data.head()

In [131]:
test_data['Age'] = test_data['Age'].replace({np.nan:mediaAgeTest})
test_data.head()

In [132]:
train_data["Edad"] = train_data['Age']

In [133]:
train_data.loc[train_data['Age'] <= 5.0, 'Edad'] = "Bebe"
train_data.loc[((train_data['Age'] >= 6.0) & (train_data['Age'] <= 14.0)), 'Edad'] = "Ninio"
train_data.loc[((train_data['Age'] >= 15.0) & (train_data['Age'] <= 30.0)), 'Edad'] = "Joven"
train_data.loc[((train_data['Age'] >= 31.0) & (train_data['Age'] <= 69.0)), 'Edad'] = "Adulto"
train_data.loc[train_data['Age'] >= 70.0, 'Edad'] = "Anciano"

In [134]:
train_data.head()

In [135]:
test_data["Edad"] = test_data['Age']

In [136]:
test_data.loc[test_data['Age'] <= 5.0, 'Edad'] = "Bebe"
test_data.loc[((test_data['Age'] >= 6.0) & (test_data['Age'] <= 14.0)), 'Edad'] = "Ninio"
test_data.loc[((test_data['Age'] >= 15.0) & (test_data['Age'] <= 30.0)), 'Edad'] = "Joven"
test_data.loc[((test_data['Age'] >= 31.0) & (test_data['Age'] <= 69.0)), 'Edad'] = "Adulto"
test_data.loc[test_data['Age'] >= 70.0, 'Edad'] = "Anciano"

In [137]:
test_data.head()

In [ ]:
#train_data = train_data.drop(["Cabin"], axis=1)
#train_data = train_data.drop(["Bebe"], axis=1)
train_data.head()

In [ ]:
#test_data = test_data.drop(["Cabin"], axis=1)
#test_data = test_data.drop(["Bebe"], axis=1)
test_data.head()

In [138]:
#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler
#from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "Relatives", "Edad" , "Embarked"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

#model = make_pipeline(StandardScaler(), SVC(gamma='auto'))
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")